In [11]:
import yaml
from src.products import load_products
from src.meals import load_meals
from src.menus import (load_menus,
                       get_bom_for_menus, calculate_total_weight, 
                       group_products_by_category, print_grouped_products)
from src.day import load_days
from src.dailyNorms import load_daily_norms

######## Change path to your files ########
menu_path = 'menu/sample_menu.yml'

green_book = chr(0x1F4D7)
blue_book = chr(0x1F4D8)
exclamation = chr(0x2757)
warning_sign = chr(0x26A0)

# Load products
products = load_products('products/products.yml')

# Load meals
meals = load_meals('meals/meals.yml', products)

# Load days
days = load_days('days/days.yml', meals)

# Load daily norms
daily_norms = load_daily_norms(menu_path)

# Load menus
menus = load_menus(menu_path, days)

# # Check rules
# print("Список уникальных дней и замечаний по ним:")
# for name, day in days.items():
#     print(f"  День: {name}, вес на человека: {day.weight:.0f} г.")
#     warnings = day.check_rules(daily_norms)
#     # Print warnings
#     for warning in warnings:
#         print(f"   {exclamation}", warning)



print(f"\n{green_book} Раскладка:")
# Print information for each menu
for i, menu in enumerate(menus, start=1):
    print(f"{green_book} Секция меню \"{menu.name}\":")
    print(f"  веса дней {[f'{day.weight:.0f}' for day in menu.days]}, общий вес {menu.total_weight:.0f}")

    # Check rules for each day in the menu
    for day in menu.days:
        warnings = day.check_rules(daily_norms)
        for warning in warnings:
            print(f"  {exclamation} {warning}")

    # Print warning count
    warning_count = sum(len(day.check_rules(daily_norms)) for day in menu.days)
    print(f"{warning_sign} {warning_count}")

    # Print meals for each day in the menu# Print meals for each day in the menu
    for j, day in enumerate(menu.days, start=1):
        print(f"  Меню на ночёвку {j}:")
        print(f"    ужин:    {day.dinner.name} ({', '.join(f'{product.name} {quantity * menu.people_count:.0f}' for product, quantity in day.dinner.products)})")
        print(f"    завтрак: {day.breakfast.name} ({', '.join(f'{product.name} {quantity * menu.people_count:.0f}' for product, quantity in day.breakfast.products)})")
        print(f"    перекус: {day.lunch.name} ({', '.join(f'{product.name} {quantity * menu.people_count:.0f}' for product, quantity in day.lunch.products)})")

    print()

# Print BOM for each menu
print(f"\n{blue_book} Список покупок для каждой секции меню по отедльности:")
for menu in menus:
    print(f"{blue_book} Меню: {menu.name}, вес: {menu.total_weight:.0f}")
    bom = get_bom_for_menus(menus)
    grouped_products = group_products_by_category(bom, products)
    print_grouped_products(grouped_products, indent=1)
    print()

print(f"\n{green_book}{green_book} Список покупок для всех меню вместе:")
bom = get_bom_for_menus(menus)
grouped_products = group_products_by_category(bom, products)
print_grouped_products(grouped_products, indent=0)
    
print(f"Общий вес: {calculate_total_weight(menus):.0f}")

Список уникальных дней и замечаний по ним:
  День: гречка-булгур, вес на человека: 740 г.
  День: булгур-пшено, вес на человека: 755 г.
   ❗ Слишком много калорий: 3321, избыток 121
  День: рис-кускус, вес на человека: 697 г.
   ❗ Слишком мало углеводов: 226, нужно ещё 64
  День: соба-макароны, вес на человека: 771 г.
   ❗ Слишком много калорий: 3374, избыток 174
  День: гречка-соба, вес на человека: 701 г.

📗 Раскладка:
📗 Секция меню "П1 дно":
  веса дней ['740', '755', '697', '771', '701'], общий вес 21986
  ❗ Слишком много калорий: 3321, избыток 121
  ❗ Слишком мало углеводов: 226, нужно ещё 64
  ❗ Слишком много калорий: 3374, избыток 174
⚠ 3
  Меню на ночёвку 1:
    ужин:    перекус гаврюшкин (Батончики сникерс 300, Кешью 150, Колбаса 300, Курага 300, Сухари 300, Сыр 300, Финики 300)
    завтрак: гречка с говядиной кронидов (Гречка 480, Пряники 300, Сухари 300, Тушенка говядина Кронидов 480)
    перекус: булгур с курятиной кронидов (Булгур 480, Козинак 240, Лук 120, Сухари 90, Туше